In [3]:
import json

with open('apikey.json') as f:
    data = json.load(f)
    api_key = data['key']
    client_id = data['client_id']
    user_agent = data['user_agent']

In [6]:
import praw

# Reddit API kimlik bilgileri
reddit = praw.Reddit(client_id=client_id,
                     client_secret=api_key,
                     user_agent=user_agent)

# Aramak istediğiniz anahtar kelime
keyword = "python"

# Subreddit belirleyin (isteğe bağlı)
subreddit_name = "learnpython"  # Örnek: "learnpython"

# Subreddit'i alın (isteğe bağlı)
subreddit = reddit.subreddit(subreddit_name)

# Belirli bir anahtar kelimeyi içeren gönderileri çekin
for submission in subreddit.search(keyword, limit=10):
    print("Başlık:", submission.title)
    print("URL:", submission.url)
    print("Oylar:", submission.score)
    print("Yorum Sayısı:", submission.num_comments)
    print("Yorumlar:")
    
    # Gönderinin yorumlarını çekin
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        print(comment.body)  # Yorum içeriğini yazdır
        print("----------------------")
    
    print("---------------------------------\n")


Başlık: Practical uses for Python for an average user ?
URL: https://www.reddit.com/r/learnpython/comments/10w01cs/practical_uses_for_python_for_an_average_user/
Oylar: 175
Yorum Sayısı: 146
Yorumlar:
One of my first ever personal python scripts was to help me with shopping. 

I manually wrote down what each aisle had at my supermarket (it was an Aldi, so not huge), and then wrote a script that would take a shopping list. 

If it could already place it to an aisle it would automatically, otherwise it would ask at the end for an aisle and remember for future use. 

The end result was I could give it a shopping list and it would give me back an ordered list of what to get in each aisle so I wasn't missing stuff and going back. 

Over time I got fancier and started integrating it with checklist apps so I could check stuff off as I went round, or integrated it with recipes, so I could be like "I'm gonna make tacos this week" and it would go "cool, in aisle one grab some peppers, in aisle t